In [49]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import catboost as cb

import optuna
from sklearn.model_selection import StratifiedGroupKFold

In [50]:
train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")

In [3]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

# シンプルに3つだけ特徴量使った場合

In [4]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# =========================================================
# 1. 前処理（例: category型を適切に指定）
# =========================================================
X = train[['HomePlanet', 'CryoSleep', 'Destination']].copy()
X_test = test[['HomePlanet', 'CryoSleep', 'Destination']].copy()


cat_cols = ['HomePlanet', 'CryoSleep', 'Destination']
for c in cat_cols:
    X[c] = X[c].astype('category')
    X_test[c] = X_test[c].astype('category')

y = train['Transported']

# =========================================================
# 2. モデルとCV設定
# =========================================================
params = {
    'objective': 'binary',
    'metric': 'accuracy',
    'n_estimators': 500,
    'learning_rate': 0.01,
    'random_state': 40
}

model = lgb.LGBMClassifier(**params)

# StratifiedKFold: ラベル比を保つ5分割
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# =========================================================
# 3. クロスバリデーションループ
# =========================================================
scores = []
for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_valid)
    acc = accuracy_score(y_valid, y_pred)
    scores.append(acc)
    print(f"Fold {fold+1} accuracy: {acc:.5f}")

# =========================================================
# 4. 平均スコア
# =========================================================
print("====================================")
print(f"Mean CV accuracy: {sum(scores)/len(scores):.5f}")


[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [5]:
model.predict(X_test)

array([ True, False,  True, ...,  True, False,  True], shape=(4277,))

In [6]:
# =========================================================
# 5. 特定条件での予測結果分析
# =========================================================
print("=== 特定条件での予測結果分析 ===")

# テストデータで予測を実行
test_predictions = model.predict(X_test)
test_predictions_proba = model.predict_proba(X_test)[:, 1]  # Transported=1の確率

# テストデータに予測結果を追加
test_with_predictions = test.copy()
test_with_predictions['Predicted_Transported'] = test_predictions
test_with_predictions['Predicted_Proba'] = test_predictions_proba

print(f"テストデータのサンプル数: {len(test)}")
print(f"予測結果 - Transported=1の数: {sum(test_predictions)}")
print(f"予測結果 - Transported=0の数: {len(test_predictions) - sum(test_predictions)}")
print(f"全体のTransported予測率: {sum(test_predictions)/len(test_predictions):.3f}")

# 条件1: 冷凍状態 + 行き先が55 Cancri e
condition1 = (test_with_predictions['CryoSleep'] == True) & (test_with_predictions['Destination'] == '55 Cancri e')
cryo_55cancri = test_with_predictions[condition1]

print(f"\n=== 条件1: 冷凍状態 + 行き先55 Cancri e ===")
print(f"該当者数: {len(cryo_55cancri)}")
if len(cryo_55cancri) > 0:
    print(f"Transported予測=1の数: {sum(cryo_55cancri['Predicted_Transported'])}")
    print(f"Transported予測=0の数: {len(cryo_55cancri) - sum(cryo_55cancri['Predicted_Transported'])}")
    print(f"Transported予測率: {sum(cryo_55cancri['Predicted_Transported'])/len(cryo_55cancri):.3f}")
    print(f"平均予測確率: {cryo_55cancri['Predicted_Proba'].mean():.3f}")
    print(f"予測確率の範囲: {cryo_55cancri['Predicted_Proba'].min():.3f} - {cryo_55cancri['Predicted_Proba'].max():.3f}")
    
    # HomePlanet別の内訳
    print("\n--- HomePlanet別の内訳 ---")
    for planet in cryo_55cancri['HomePlanet'].unique():
        planet_data = cryo_55cancri[cryo_55cancri['HomePlanet'] == planet]
        if len(planet_data) > 0:
            print(f"{planet}: {len(planet_data)}人, Transported予測率: {sum(planet_data['Predicted_Transported'])/len(planet_data):.3f}")
else:
    print("該当者なし")

# 条件2: 冷凍状態 + HomePlanet=Europa
condition2 = (test_with_predictions['CryoSleep'] == True) & (test_with_predictions['HomePlanet'] == 'Europa')
cryo_europa = test_with_predictions[condition2]

print(f"\n=== 条件2: 冷凍状態 + HomePlanet=Europa ===")
print(f"該当者数: {len(cryo_europa)}")
if len(cryo_europa) > 0:
    print(f"Transported予測=1の数: {sum(cryo_europa['Predicted_Transported'])}")
    print(f"Transported予測=0の数: {len(cryo_europa) - sum(cryo_europa['Predicted_Transported'])}")
    print(f"Transported予測率: {sum(cryo_europa['Predicted_Transported'])/len(cryo_europa):.3f}")
    print(f"平均予測確率: {cryo_europa['Predicted_Proba'].mean():.3f}")
    print(f"予測確率の範囲: {cryo_europa['Predicted_Proba'].min():.3f} - {cryo_europa['Predicted_Proba'].max():.3f}")
    
    # Destination別の内訳
    print("\n--- Destination別の内訳 ---")
    for dest in cryo_europa['Destination'].unique():
        dest_data = cryo_europa[cryo_europa['Destination'] == dest]
        if len(dest_data) > 0:
            print(f"{dest}: {len(dest_data)}人, Transported予測率: {sum(dest_data['Predicted_Transported'])/len(dest_data):.3f}")
else:
    print("該当者なし")

# 条件3: 冷凍状態 + HomePlanet=Europa + Destination=55 Cancri e（両方の条件を満たす）
condition3 = (test_with_predictions['CryoSleep'] == True) & \
             (test_with_predictions['HomePlanet'] == 'Europa') & \
             (test_with_predictions['Destination'] == '55 Cancri e')
cryo_europa_55cancri = test_with_predictions[condition3]

print(f"\n=== 条件3: 冷凍状態 + Europa + 55 Cancri e ===")
print(f"該当者数: {len(cryo_europa_55cancri)}")
if len(cryo_europa_55cancri) > 0:
    print(f"Transported予測=1の数: {sum(cryo_europa_55cancri['Predicted_Transported'])}")
    print(f"Transported予測=0の数: {len(cryo_europa_55cancri) - sum(cryo_europa_55cancri['Predicted_Transported'])}")
    print(f"Transported予測率: {sum(cryo_europa_55cancri['Predicted_Transported'])/len(cryo_europa_55cancri):.3f}")
    print(f"平均予測確率: {cryo_europa_55cancri['Predicted_Proba'].mean():.3f}")
else:
    print("該当者なし")

# 参考: 冷凍状態全体の予測結果
cryo_all = test_with_predictions[test_with_predictions['CryoSleep'] == True]
print(f"\n=== 参考: 冷凍状態全体 ===")
print(f"冷凍状態の人数: {len(cryo_all)}")
print(f"Transported予測率: {sum(cryo_all['Predicted_Transported'])/len(cryo_all):.3f}")
print(f"平均予測確率: {cryo_all['Predicted_Proba'].mean():.3f}")

=== 特定条件での予測結果分析 ===
テストデータのサンプル数: 4277
予測結果 - Transported=1の数: 1584
予測結果 - Transported=0の数: 2693
全体のTransported予測率: 0.370

=== 条件1: 冷凍状態 + 行き先55 Cancri e ===
該当者数: 364
Transported予測=1の数: 364
Transported予測=0の数: 0
Transported予測率: 1.000
平均予測確率: 0.927
予測確率の範囲: 0.755 - 0.997

--- HomePlanet別の内訳 ---
Europa: 217人, Transported予測率: 1.000
Earth: 94人, Transported予測率: 1.000
Mars: 44人, Transported予測率: 1.000

=== 条件2: 冷凍状態 + HomePlanet=Europa ===
該当者数: 463
Transported予測=1の数: 463
Transported予測=0の数: 0
Transported予測率: 1.000
平均予測確率: 0.987
予測確率の範囲: 0.978 - 0.997

--- Destination別の内訳 ---
55 Cancri e: 217人, Transported予測率: 1.000
TRAPPIST-1e: 230人, Transported予測率: 1.000
PSO J318.5-22: 7人, Transported予測率: 1.000

=== 条件3: 冷凍状態 + Europa + 55 Cancri e ===
該当者数: 217
Transported予測=1の数: 217
Transported予測=0の数: 0
Transported予測率: 1.000
平均予測確率: 0.997

=== 参考: 冷凍状態全体 ===
冷凍状態の人数: 1544
Transported予測率: 1.000
平均予測確率: 0.817


## できるだけ多くの特徴量使った場合

In [ ]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """欠損値を補完せずに特徴量を生成する"""
    data = df.copy()

    # Cabin情報を分解（欠損はNaNのまま保持）
    if 'Cabin' in data.columns:
        cabin_split = data['Cabin'].str.split('/', expand=True)
        data['Deck'] = cabin_split[0]
        data['Num'] = pd.to_numeric(cabin_split[1], errors='coerce')  # NaNそのまま
        data['Side'] = cabin_split[2]
    else:
        data['Deck'] = np.nan
        data['Num'] = np.nan
        data['Side'] = np.nan

    # 支出関連特徴量（NaNはそのまま残す）
    expense_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in expense_cols:
        if col not in data.columns:
            data[col] = np.nan

    # 総支出額（NaNのまま計算 → NaNを含む場合はNaN）
    data['TotalExpense'] = data[expense_cols].sum(axis=1, skipna=False)

    # 支出が一つでもあるか（NaNを無視）
    data['HasExpense'] = data[expense_cols].gt(0).any(axis=1).astype(int)

    # 年齢グループ（NaNは'Unknown'にせず、NaNのまま）
    bins = [0, 12, 18, 30, 50, 100]
    labels = ['Child', 'Teen', 'Young', 'Adult', 'Senior']
    data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels)

    # Name情報から家族サイズを生成（欠損はNaN）
    if 'Name' in data.columns:
        data['FamilyName'] = data['Name'].str.split().str[-1]
        family_counts = data['FamilyName'].value_counts()
        data['FamilySize'] = data['FamilyName'].map(family_counts)
        data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    else:
        data['FamilyName'] = np.nan
        data['FamilySize'] = np.nan
        data['IsAlone'] = np.nan

    # PassengerId情報（欠損時はNaN）
    if 'PassengerId' in data.columns:
        passenger_split = data['PassengerId'].str.split('_', expand=True)
        data['PassengerGroup'] = passenger_split[0]
        data['PassengerNum'] = pd.to_numeric(passenger_split[1], errors='coerce')
    else:
        data['PassengerGroup'] = np.nan
        data['PassengerNum'] = np.nan

    return data

In [8]:

# =========================================================
# 2. 特徴量生成
# =========================================================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
    'Deck', 'Side', 'Num',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'TotalExpense', 'HasExpense', 'AgeGroup', 'FamilySize', 'IsAlone'
]

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()
y_full = train['Transported']

# カテゴリ列をcategory型に変換（NaNも保持）
categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Deck', 'Side', 'AgeGroup'
]

for col in categorical_features:
    X_full[col] = X_full[col].astype('category')
    X_test_full[col] = X_test_full[col].astype('category')

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

# =========================================================
# 3. LightGBM設定 + 5分割CV
# =========================================================
params_full = {
    'objective': 'binary',
    'metric': 'accuracy',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 42,
    'verbose': -1,
}

cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_full = []
feature_importance_list = []

print("\n=== 5-Fold Cross Validation (欠損値未補完版) ===")
for fold, (train_idx, valid_idx) in enumerate(cv_full.split(X_full, y_full)):
    X_train_full, X_valid_full = X_full.iloc[train_idx], X_full.iloc[valid_idx]
    y_train_full, y_valid_full = y_full.iloc[train_idx], y_full.iloc[valid_idx]

    model = lgb.LGBMClassifier(**params_full)
    model.fit(X_train_full, y_train_full)

    y_pred_full = model.predict(X_valid_full)
    acc_full = accuracy_score(y_valid_full, y_pred_full)
    scores_full.append(acc_full)

    feature_importance_list.append(model.feature_importances_)

    print(f"Fold {fold+1} accuracy: {acc_full:.5f}")

# =========================================================
# 4. 結果まとめ
# =========================================================
print("=" * 50)
print(f"Mean CV accuracy (欠損未補完): {np.mean(scores_full):.5f}")
print(f"Standard deviation: {np.std(scores_full):.5f}")

feature_importance_avg = np.mean(feature_importance_list, axis=0)
feature_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance_avg
}).sort_values('importance', ascending=False)

print("\n=== Top 10 重要特徴量 ===")
print(feature_importance_df.head(10).to_string(index=False))

# =========================================================
# 5. 全データで再学習
# =========================================================
print("\n=== 全データで最終モデルを学習 ===")
final_model = lgb.LGBMClassifier(**params_full)
final_model.fit(X_full, y_full)
print("最終モデル学習完了")


使用特徴量数: 18
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'Deck', 'Side', 'Num', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalExpense', 'HasExpense', 'AgeGroup', 'FamilySize', 'IsAlone']
データシェイプ: Train (8693, 18), Test (4277, 18)

=== 5-Fold Cross Validation (欠損値未補完版) ===
Fold 1 accuracy: 0.80851
Fold 2 accuracy: 0.80679
Fold 3 accuracy: 0.80449
Fold 4 accuracy: 0.80437
Fold 5 accuracy: 0.79287
Mean CV accuracy (欠損未補完): 0.80340
Standard deviation: 0.00549

=== Top 10 重要特徴量 ===
     feature  importance
         Num      5549.2
         Age      3766.8
TotalExpense      3067.0
  FamilySize      2313.6
   FoodCourt      1945.2
ShoppingMall      1912.8
         Spa      1891.4
      VRDeck      1856.8
 RoomService      1838.2
 Destination       727.8

=== 全データで最終モデルを学習 ===
最終モデル学習完了


## 特徴量を吟味した場合

最終的にここにある特徴量で出してる
feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination',  'VIP',
    'Side', 'Num',
    'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck',
    'TotalExpense', 'AgeGroup'
]



In [51]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """欠損値を補完せずに特徴量を生成する"""
    data = df.copy()

    # Cabin情報を分解（欠損はNaNのまま保持）
    if 'Cabin' in data.columns:
        cabin_split = data['Cabin'].str.split('/', expand=True)
        data['Deck'] = cabin_split[0]
        data['Num'] = pd.to_numeric(cabin_split[1], errors='coerce')  # NaNそのまま
        data['Side'] = cabin_split[2]
    else:
        data['Deck'] = np.nan
        data['Num'] = np.nan
        data['Side'] = np.nan

    # 支出関連特徴量（NaNはそのまま残す）
    expense_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in expense_cols:
        if col not in data.columns:
            data[col] = np.nan

    # 総支出額（NaNのまま計算 → NaNを含む場合はNaN）
    data['TotalExpense'] = data[expense_cols].sum(axis=1, skipna=False)

    # 支出が一つでもあるか（NaNを無視）
    data['HasExpense'] = data[expense_cols].gt(0).any(axis=1).astype(int)

    # 年齢グループ（NaNは'Unknown'にせず、NaNのまま）
    bins = [0, 12, 18, 30, 50, 100]
    labels = ['Child', 'Teen', 'Young', 'Adult', 'Senior']
    data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels)

    # Name情報から家族サイズを生成（欠損はNaN）
    if 'Name' in data.columns:
        data['FamilyName'] = data['Name'].str.split().str[-1]
        family_counts = data['FamilyName'].value_counts()
        data['FamilySize'] = data['FamilyName'].map(family_counts)
        data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    else:
        data['FamilyName'] = np.nan
        data['FamilySize'] = np.nan
        data['IsAlone'] = np.nan

    # PassengerId情報（欠損時はNaN）
    if 'PassengerId' in data.columns:
        passenger_split = data['PassengerId'].str.split('_', expand=True)
        data['PassengerGroup'] = passenger_split[0]
        data['PassengerNum'] = pd.to_numeric(passenger_split[1], errors='coerce')
    else:
        data['PassengerGroup'] = np.nan
        data['PassengerNum'] = np.nan

    return data

In [17]:

# =========================================================
# 2. 特徴量生成
# =========================================================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination',  'VIP',
    'Side', 'Num',
    'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck',
    'TotalExpense', 'AgeGroup'
]
# 'Age', 'Deck', 'FamilySize', 'IsAlone', 'HasExpense',

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()
y_full = train['Transported']

# カテゴリ列をcategory型に変換（NaNも保持）
categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'AgeGroup'
]

for col in categorical_features:
    X_full[col] = X_full[col].astype('category')
    X_test_full[col] = X_test_full[col].astype('category')

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

# =========================================================
# 3. LightGBM設定 + 5分割CV
# =========================================================
params_full = {
    'objective': 'binary',
    'metric': 'accuracy',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 42,
    'verbose': -1,
}

cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_full = []
feature_importance_list = []

print("\n=== 5-Fold Cross Validation (欠損値未補完版) ===")
for fold, (train_idx, valid_idx) in enumerate(cv_full.split(X_full, y_full)):
    X_train_full, X_valid_full = X_full.iloc[train_idx], X_full.iloc[valid_idx]
    y_train_full, y_valid_full = y_full.iloc[train_idx], y_full.iloc[valid_idx]

    model = lgb.LGBMClassifier(**params_full)
    model.fit(X_train_full, y_train_full)

    y_pred_full = model.predict(X_valid_full)
    acc_full = accuracy_score(y_valid_full, y_pred_full)
    scores_full.append(acc_full)

    feature_importance_list.append(model.feature_importances_)

    print(f"Fold {fold+1} accuracy: {acc_full:.5f}")

# =========================================================
# 4. 結果まとめ
# =========================================================
print("=" * 50)
print(f"Mean CV accuracy (欠損未補完): {np.mean(scores_full):.5f}")
print(f"Standard deviation: {np.std(scores_full):.5f}")

feature_importance_avg = np.mean(feature_importance_list, axis=0)
feature_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance_avg
}).sort_values('importance', ascending=False)

print("\n=== Top 10 重要特徴量 ===")
print(feature_importance_df.head(10).to_string(index=False))

# =========================================================
# 5. 全データで再学習
# =========================================================
print("\n=== 全データで最終モデルを学習 ===")
final_model = lgb.LGBMClassifier(**params_full)
final_model.fit(X_full, y_full)
print("最終モデル学習完了")


使用特徴量数: 13
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalExpense', 'AgeGroup']
データシェイプ: Train (8693, 13), Test (4277, 13)

=== 5-Fold Cross Validation (欠損値未補完版) ===
Fold 1 accuracy: 0.79988
Fold 2 accuracy: 0.79298
Fold 3 accuracy: 0.79356
Fold 4 accuracy: 0.79689
Fold 5 accuracy: 0.78769
Mean CV accuracy (欠損未補完): 0.79420
Standard deviation: 0.00410

=== Top 10 重要特徴量 ===
     feature  importance
         Num      4254.0
TotalExpense      2620.6
      VRDeck      1814.6
ShoppingMall      1813.6
   FoodCourt      1794.2
         Spa      1776.2
 RoomService      1712.6
    AgeGroup      1024.2
  HomePlanet       580.2
        Side       542.0

=== 全データで最終モデルを学習 ===
最終モデル学習完了


In [18]:

def objective(trial: optuna.Trial) -> float:
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_estimators': 600,
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 40),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 120),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log=True),
        'random_state': 42,
        # ← 重要: metricはfitで与えるのでパラメータ側では無効化
        'metric': 'None',
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)

        callbacks = [
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0),
        ]

        # 重要: eval_metric は 'binary_logloss' か 'auc' など公式名を指定
        model.fit(
            X_trn, y_trn,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            categorical_feature=categorical_features,
            callbacks=callbacks,
        )

        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# Optuna 実行（安定のTPE＋MedianPruner）
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# 最終学習（全データ）: early stoppingは使わない
best_params = study.best_params | {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'n_estimators': 600,
    'metric': 'None',
    'random_state': 42,
}
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_full, y_full, categorical_feature=categorical_features)
print("\n最終モデル学習完了")


[I 2025-10-13 21:39:42,815] A new study created in memory with name: no-name-38e2753d-f34d-430f-aad5-d3647cc6e155
Best trial: 0. Best value: 0.804785:   2%|▎         | 1/40 [00:03<01:59,  3.06s/it]

[I 2025-10-13 21:39:45,877] Trial 0 finished with value: 0.804784570580423 and parameters: {'learning_rate': 0.047377279007281566, 'num_leaves': 39, 'max_depth': 5, 'min_data_in_leaf': 76, 'feature_fraction': 0.7468055921327309, 'bagging_fraction': 0.7467983561008608, 'bagging_freq': 1, 'lambda_l1': 0.39676050770529864, 'lambda_l2': 0.06358358856676251}. Best is trial 0 with value: 0.804784570580423.


Best trial: 1. Best value: 0.80651:   5%|▌         | 2/40 [00:04<01:24,  2.22s/it] 

[I 2025-10-13 21:39:47,514] Trial 1 finished with value: 0.8065095014462104 and parameters: {'learning_rate': 0.10211806418789511, 'num_leaves': 15, 'max_depth': 6, 'min_data_in_leaf': 102, 'feature_fraction': 0.7637017332034828, 'bagging_fraction': 0.7545474901621302, 'bagging_freq': 1, 'lambda_l1': 0.008179499475211672, 'lambda_l2': 0.03752055855124281}. Best is trial 1 with value: 0.8065095014462104.


Best trial: 1. Best value: 0.80651:   8%|▊         | 3/40 [00:07<01:33,  2.52s/it]

...

Best trial: 1. Best value: 0.80651:  18%|█▊        | 7/40 [00:14<00:53,  1.63s/it]

[I 2025-10-13 21:39:57,251] Trial 6 pruned. 


Best trial: 1. Best value: 0.80651:  18%|█▊        | 7/40 [00:16<01:16,  2.31s/it]

[W 2025-10-13 21:39:58,967] Trial 7 failed with parameters: {'learning_rate': 0.04547611147126364, 'num_leaves': 22, 'max_depth': 5, 'min_data_in_leaf': 25, 'feature_fraction': 0.9406590942262119, 'bagging_fraction': 0.7223651931039312, 'bagging_freq': 5, 'lambda_l1': 0.20736445177905022, 'lambda_l2': 0.0039459088110999985} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/yhashiguchi/Library/Mobile Documents/com~apple~CloudDocs/Desktop/3A/kaggle/spaceship-titanic-ut-komaba-2025/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/8x/_6ksv3c137v05nnyhyck18xw0000gn/T/ipykernel_99017/3335615857.py", line 36, in objective
    model.fit(
  File "/Users/yhashiguchi/Library/Mobile Documents/com~apple~CloudDocs/Desktop/3A/kaggle/spaceship-titanic-ut-komaba-2025/.venv/lib/python3.12/site-packages/lightgbm/sklearn.py", li

KeyboardInterrupt: 

## 家族を同じCV splitに

In [22]:
def objective(trial: optuna.Trial) -> float:
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_estimators': 600,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log=True),
        'random_state': 42,
        'metric': 'None',  # metricはfitで与える
    }

    # --- 重要: 同じ苗字の人を同じsplitに入れる ---
    # create_featuresで作った FamilyName列を使う
    groups = train_features['FamilyName'].fillna('Unknown').astype(str)

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        callbacks = [
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0),
        ]

        model.fit(
            X_trn, y_trn,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',  # LightGBM標準指標
            categorical_feature=categorical_features,
            callbacks=callbacks,
        )

        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# --- Optuna設定 ---
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# --- 最終モデル学習 ---
best_params = study.best_params | {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'n_estimators': 600,
    'metric': 'None',
    'random_state': 42,
}
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_full, y_full, categorical_feature=categorical_features)
print("\n最終モデル学習完了")


[I 2025-10-13 21:49:29,645] A new study created in memory with name: no-name-c5d13773-48b5-41e5-9eaf-c607291e117f
Best trial: 0. Best value: 0.802922:   2%|▎         | 1/40 [00:04<02:58,  4.57s/it]

[I 2025-10-13 21:49:34,214] Trial 0 finished with value: 0.802922091220713 and parameters: {'learning_rate': 0.02757359293934948, 'num_leaves': 49, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.7468055921327309, 'bagging_fraction': 0.7467983561008608, 'bagging_freq': 1, 'lambda_l1': 0.39676050770529864, 'lambda_l2': 0.06358358856676251}. Best is trial 0 with value: 0.802922091220713.


Best trial: 1. Best value: 0.803543:   5%|▌         | 2/40 [00:06<02:04,  3.28s/it]

[I 2025-10-13 21:49:36,593] Trial 1 finished with value: 0.8035430110705348 and parameters: {'learning_rate': 0.06803900745073703, 'num_leaves': 15, 'max_depth': 6, 'min_data_in_leaf': 85, 'feature_fraction': 0.7637017332034828, 'bagging_fraction': 0.7545474901621302, 'bagging_freq': 1, 'lambda_l1': 0.008179499475211672, 'lambda_l2': 0.03752055855124281}. Best is trial 1 with value: 0.8035430110705348.


Best trial: 1. Best value: 0.803543:   8%|▊         | 3/40 [00:11<02:28,  4.02s/it]

...

[I 2025-10-13 21:51:14,783] Trial 37 finished with value: 0.8045717594119767 and parameters: {'learning_rate': 0.036854677424362714, 'num_leaves': 23, 'max_depth': 4, 'min_data_in_leaf': 58, 'feature_fraction': 0.7904264012703551, 'bagging_fraction': 0.7407854098178079, 'bagging_freq': 3, 'lambda_l1': 0.009937477282014226, 'lambda_l2': 0.004205315535809935}. Best is trial 21 with value: 0.8063753479314061.


Best trial: 21. Best value: 0.806375:  98%|█████████▊| 39/40 [01:47<00:02,  2.48s/it]

[I 2025-10-13 21:51:17,098] Trial 38 finished with value: 0.8044057362907233 and parameters: {'learning_rate': 0.045163167568468936, 'num_leaves': 36, 'max_depth': 3, 'min_data_in_leaf': 23, 'feature_fraction': 0.751274542648332, 'bagging_fraction': 0.8383862690694426, 'bagging_freq': 5, 'lambda_l1': 0.4024420022771972, 'lambda_l2': 0.18907065196835945}. Best is trial 21 with value: 0.8063753479314061.


Best trial: 21. Best value: 0.806375: 100%|██████████| 40/40 [01:50<00:00,  2.76s/it]


[I 2025-10-13 21:51:20,069] Trial 39 finished with value: 0.8039421199640089 and parameters: {'learning_rate': 0.028857483071269386, 'num_leaves': 43, 'max_depth': 3, 'min_data_in_leaf': 53, 'feature_fraction': 0.850160405382329, 'bagging_fraction': 0.856906975963815, 'bagging_freq': 4, 'lambda_l1': 0.8731417894973494, 'lambda_l2': 0.011039246558388283}. Best is trial 21 with value: 0.8063753479314061.

=== Optuna 結果 ===
Best CV accuracy: 0.80638
learning_rate: 0.04751119756724808
num_leaves: 41
max_depth: 3
min_data_in_leaf: 12
feature_fraction: 0.7870889818820944
bagging_fraction: 0.8593516788448047
bagging_freq: 4
lambda_l1: 0.01791397438191837
lambda_l2: 0.009864085574130065

最終モデル学習完了


In [45]:

# =========================================================
# 2. 特徴量生成
# =========================================================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination',  'VIP',
    'Side', 'Num',
    'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck',
    'TotalExpense', 'AgeGroup'
]
# 'Age', 'Deck', 'FamilySize', 'IsAlone', 'HasExpense',

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()
y_full = train['Transported']

# カテゴリ列をcategory型に変換（NaNも保持）
categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'AgeGroup'
]

for col in categorical_features:
    X_full[col] = X_full[col].astype('category')
    X_test_full[col] = X_test_full[col].astype('category')

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

# =========================================================
# 3. LightGBM設定 + 5分割CV
# =========================================================
params_full = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_estimators': 600,
        'learning_rate': 0.04751119756724808,
        'num_leaves': 41,
        'max_depth': 3,
        'min_data_in_leaf': 12,
        'feature_fraction': 0.7870889818820944,
        'bagging_fraction': 0.8593516788448047,
        'bagging_freq': 4,
        'lambda_l1': 0.01791397438191837,
        'lambda_l2': 0.009864085574130065,
        'random_state': 42,
        'metric': 'None'
    }

cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_full = []
feature_importance_list = []

# --- 重要: 同じ苗字の人を同じsplitに入れる ---
# create_featuresで作った FamilyName列を使う
groups = train_features['FamilyName'].fillna('Unknown').astype(str)

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=420)
fold_scores = []

print("\n=== 5-Fold Cross Validation (欠損値未補完版) ===")
for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
    X_train_full, X_valid_full = X_full.iloc[trn_idx], X_full.iloc[val_idx]
    y_train_full, y_valid_full = y_full.iloc[trn_idx], y_full.iloc[val_idx]

    model = lgb.LGBMClassifier(**params_full)
    model.fit(X_train_full, y_train_full)

    y_pred_full = model.predict(X_valid_full)
    acc_full = accuracy_score(y_valid_full, y_pred_full)
    scores_full.append(acc_full)

    feature_importance_list.append(model.feature_importances_)

    print(f"Fold {fold_idx} accuracy: {acc_full:.5f}")

# =========================================================
# 4. 結果まとめ
# =========================================================
print("=" * 50)
print(f"Mean CV accuracy (欠損未補完): {np.mean(scores_full):.5f}")
print(f"Standard deviation: {np.std(scores_full):.5f}")

feature_importance_avg = np.mean(feature_importance_list, axis=0)
feature_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance_avg
}).sort_values('importance', ascending=False)

print("\n=== Top 10 重要特徴量 ===")
print(feature_importance_df.head(10).to_string(index=False))

# =========================================================
# 5. 全データで再学習
# =========================================================
print("\n=== 全データで最終モデルを学習 ===")
final_model = lgb.LGBMClassifier(**params_full)
final_model.fit(X_full, y_full)
print("最終モデル学習完了")


使用特徴量数: 13
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalExpense', 'AgeGroup']
データシェイプ: Train (8693, 13), Test (4277, 13)

=== 5-Fold Cross Validation (欠損値未補完版) ===
Fold 1 accuracy: 0.81371
Fold 1 accuracy: 0.81371
Fold 2 accuracy: 0.80496
Fold 2 accuracy: 0.80496
Fold 3 accuracy: 0.82654
Fold 3 accuracy: 0.82654
Fold 4 accuracy: 0.79461
Fold 4 accuracy: 0.79461
Fold 5 accuracy: 0.78620
Mean CV accuracy (欠損未補完): 0.80521
Standard deviation: 0.01415

=== Top 10 重要特徴量 ===
     feature  importance
         Num       728.4
      VRDeck       452.6
         Spa       449.2
   FoodCourt       418.4
TotalExpense       413.6
 RoomService       365.8
ShoppingMall       333.2
  HomePlanet       228.0
    AgeGroup       138.4
        Side       117.2

=== 全データで最終モデルを学習 ===
Fold 5 accuracy: 0.78620
Mean CV accuracy (欠損未補完): 0.80521
Standard deviation: 0.01415

=== Top 10 重要特徴量 ===
     feature  importance
 

In [46]:
# 全データで学習した場合。出力まで行う

final_model = lgb.LGBMClassifier(**params_full)
final_model.fit(X_full, y_full)

preds = final_model.predict(X_test_full)


passenger_ids = test["PassengerId"]

# DataFrame にまとめる
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Transported": preds
})

# CSV 出力（インデックスは不要）
submission.to_csv("submission.csv", index=False)

In [47]:
submission['Transported'].value_counts()

Transported
True     2223
False    2054
Name: count, dtype: int64